In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import os
import numpy as np

In [ ]:
def image_files(path):
    '''
    Creates a list with the names of the images to be processed.
    '''
    image_files = []
    # creates a ScandirIterator aliased as files
    with os.scandir(path) as files:
        for file in files:
            if file.name.endswith('.png'):
                # adds only the image files to the list
                image_files.append(file.name)
    return image_files


def load_model():
    model = VGG16()
    model = Model(inputs = model.inputs, outputs = model.output)
    return model


def recognize_objects(file, path, model):
    '''
    Recognizes objects in an image file using the VGG16 model.
    '''
    # load the image as a 224x224 array
    img = load_img(f'{path}/{file}', target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img)
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3)
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get object names
    objects = model.predict(imgx)
    return objects

In [ ]:
path = "./data"
model = load_model()
files = image_files(path)

for file in files: 
    objects= recognize_objects(file, path, model)
    print (f'Objects detected in {file} are:')
    print (objects)